In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import ipywidgets as widgets
from elm_data.step_5_label_elms.elm_labeling_app import BES_ELM_Labeling_App
from elm_data.data_tools import print_hdf5_contents

In [ ]:
plt.close('all')
widgets.Widget.close_all()

elm_label_file = './labeled_elms_v2.hdf5'

app = BES_ELM_Labeling_App(
    data_hdf5_file='/home/smithdr/ml/elm_data/step_4_shot_partial_data/data_v5.hdf5',
    elm_labels_hdf5_file=elm_label_file,
    truncate=True,
)

In [ ]:
widgets.AppLayout(
    left_sidebar=app.controls,
    center=app.fig.canvas,
    footer=app.status_label,
    pane_widths=['160px', 1, 0],
    pane_heights=[0, 1, '40px'],
)

In [ ]:
print_hdf5_contents(elm_label_file, print_attributes=False, print_datasets=False)